# Tutorial \#3: Blach Hole and Accretion Disk Parameters

**Nicolás Guerra-Varas**
Professor Dragana Ilić
Tutor Isidora Jankov

Date: 

Data: sample of 500 SDSS quasars (QSOs) with spectral parameters from __[Shen et al. 2011
](https://ui.adsabs.harvard.edu/abs/2011ApJS..194...45S/abstract)__ (__[VizieR catalog](https://vizier.cds.unistra.fr/viz-bin/VizieR?-source=J/ApJS/194/45)__).

The goal of this tutorial is to learn how to estimate the mass of the black hole (BH) of active galactic nuclei (AGN) with indirect methods: scaling relationships from reverberation mapping (RM) and the use of the bulge luminosity of the host galaxy.

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [21]:
qsos = pd.read_csv('shen_subsample_500.csv')
qsos.reset_index(inplace=True, drop=True)
qsos

,RAJ2000,DEJ2000,Plate,Fiber,MJD,iMAG,logLbol,e_logLbol,logLBHb,e_logLBHb,W_BHb_,e_W_BHb_,EWBHb,e_EWBHb,SN_Hb_,logBH,e_logBH,logEdd
0,189.67804,45.30687,1372,486,53062,-23.20,45.236,0.004,42.35,0.02,4399.3,330.1,62.0,3.1,23.06,8.33,0.07,-1.20
1,189.77188,41.44564,1984,609,53433,-22.29,44.819,0.005,42.11,0.04,2371.4,541.6,89.4,8.7,15.06,7.59,0.20,-0.87
2,189.81419,53.23740,885,449,52379,-22.69,45.121,0.004,41.93,0.04,10756.6,4366.4,32.1,3.4,16.27,9.05,0.35,-2.03
3,189.93733,19.90707,2614,350,54481,-23.37,45.132,0.001,42.55,0.02,15241.3,362.8,118.7,6.3,19.79,9.36,0.02,-2.33
4,189.99405,49.09448,1278,96,52735,-22.77,45.166,0.011,42.25,0.01,1385.6,48.6,52.5,2.9,15.96,7.29,0.03,-0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,231.08040,27.95578,1846,28,54173,-23.33,45.423,0.005,42.56,0.02,9345.2,348.3,60.6,3.4,24.76,9.08,0.03,-1.76
496,231.09229,26.70657,2160,444,53885,-22.72,44.989,0.005,42.19,0.04,4363.7,503.4,73.2,6.8,17.34,8.20,0.10,-1.31
497,231.11946,30.54376,1650,545,53174,-24.56,45.891,0.001,43.14,0.01,2572.5,106.9,76.1,1.9,40.15,8.19,0.04,-0.40
498,231.12225,35.25613,1354,466,52814,-22.60,45.143,0.005,41.80,0.05,1559.7,243.2,21.0,2.7,15.44,7.38,0.14,-0.34


In [22]:
qsos.columns

Index(['RAJ2000', 'DEJ2000', 'Plate', 'Fiber', 'MJD', 'iMAG', 'logLbol',
       'e_logLbol', 'logLBHb', 'e_logLBHb', 'W_BHb_', 'e_W_BHb_', 'EWBHb',
       'e_EWBHb', 'SN_Hb_', 'logBH', 'e_logBH', 'logEdd'],
      dtype='object')

In [23]:
qsos.shape

(500, 18)

## Scaling relations from RM

Get txt file with plate, mjd and fiber ID for SDSS cross-match

In [30]:
cross_id = qsos[['Plate', 'MJD', 'Fiber']]
cross_id

,Plate,MJD,Fiber
0,1372,53062,486
1,1984,53433,609
2,885,52379,449
3,2614,54481,350
4,1278,52735,96
...,...,...,...
495,1846,54173,28
496,2160,53885,444
497,1650,53174,545
498,1354,52814,466


In [32]:
qsos.shape

(500, 18)

In [33]:
cross_id.shape

(500, 3)

In [43]:
Counter(cross_id.duplicated())

Counter({False: 500})

In [36]:
cross_id_str = cross_id.to_string(index=False, header=False)
cross_id_file = open('cross_id_input.txt', 'a')
cross_id_file.write(cross_id_str)
cross_id_file.close()

In [58]:
cross_id.to_csv('cross_id_input.csv', index=False, header=False)

I used the `cross_id_input.txt` file to cross match the sample with SDSS in the __[CrossMatch Tool](https://skyserver.sdss.org/dr18/CrossMatchTools/ObjectCrossID)__. The SQL query I used is:

```
SELECT s.z AS redshift, g.h_beta_flux
FROM #upload u
      JOIN SpecObjAll as s
      ON (s.plate=u.up_plate AND s.mjd=u.up_mjd AND s.fiberID=u.up_fiber)
      JOIN
      GalSpecLine AS g
      ON g.specobjid = s.specobjid
```

The result is in the `crosmatch_shen_subsample.csv` file.

In [60]:
sdss_qsos = pd.read_csv('crosmatch_shen_subsample.csv', skiprows=1)
sdss_qsos

,redshift,h_beta_flux,ra,dec,plate,mjd,fiberID
0,0.183985,300.96690,195.03333,3.932381,849,52439,300
1,0.252761,391.48320,217.41974,3.357194,585,52027,214
2,0.268422,410.67200,219.91783,3.091284,586,52023,89
3,0.192150,199.62830,201.07607,58.819914,959,52411,192
4,0.171041,153.48380,203.15810,59.514940,960,52425,321
...,...,...,...,...,...,...,...
495,0.250829,15.71955,200.50446,51.710448,1667,53430,543
496,0.258147,525.93010,221.39478,46.643204,1675,53466,428
497,0.227546,210.10160,205.87458,28.055933,2017,53474,180
498,0.211458,172.48770,205.95884,28.034967,2017,53474,168


In [61]:
sdss_qsos.columns

Index(['redshift', 'h_beta_flux', 'ra', 'dec', 'plate', 'mjd', 'fiberID'], dtype='object')

In [62]:
sdss_qsos['ra'].values

array([195.03333, 217.41974, 219.91783, 201.07607, 203.1581 , 212.88279,
       225.72551, 220.15972, 204.95997, 210.14029, 209.73916, 215.66096,
       215.51932, 203.63985, 218.73048, 207.01705, 215.80913, 228.43055,
       195.71049, 212.93848, 223.49888, 196.78653, 219.83046, 224.74711,
       224.8786 , 205.14716, 201.32559, 205.15922, 209.33116, 222.10458,
       202.71351, 204.48093, 206.95824, 206.95028, 218.59573, 229.36864,
       229.90061, 225.49625, 196.16958, 205.27304, 196.23714, 205.6301 ,
       212.81968, 227.30369, 195.26462, 198.01962, 194.41389, 189.77188,
       203.6345 , 194.53106, 211.0299 , 192.92617, 228.32586, 228.15441,
       230.52113, 204.80162, 206.31885, 189.67803, 191.8823 , 203.59711,
       207.22821, 215.85612, 199.78264, 198.28565, 191.2451 , 196.08887,
       199.35564, 191.58713, 193.08148, 195.22941, 204.86595, 220.11897,
       196.96941, 197.98421, 199.45968, 197.07876, 207.36907, 197.15658,
       210.40265, 190.92405, 210.87303, 202.81516, 

In [63]:
qsos.loc[qsos['RAJ2000']==sdss_qsos['ra'].values]

,RAJ2000,DEJ2000,Plate,Fiber,MJD,iMAG,logLbol,e_logLbol,logLBHb,e_logLBHb,W_BHb_,e_W_BHb_,EWBHb,e_EWBHb,SN_Hb_,logBH,e_logBH,logEdd
453,226.54376,2.28056,539,586,52017,-23.43,45.249,0.001,42.47,0.01,3265.3,98.2,72.5,1.5,47.45,8.08,0.03,-0.93


In [17]:
qsos

,RAJ2000,DEJ2000,Plate,Fiber,MJD,iMAG,logLbol,e_logLbol,logLBHb,e_logLBHb,W_BHb_,e_W_BHb_,EWBHb,e_EWBHb,SN_Hb_,logBH,e_logBH,logEdd
0,189.67804,45.30687,1372,486,53062,-23.20,45.236,0.004,42.35,0.02,4399.3,330.1,62.0,3.1,23.06,8.33,0.07,-1.20
1,189.77188,41.44564,1984,609,53433,-22.29,44.819,0.005,42.11,0.04,2371.4,541.6,89.4,8.7,15.06,7.59,0.20,-0.87
2,189.81419,53.23740,885,449,52379,-22.69,45.121,0.004,41.93,0.04,10756.6,4366.4,32.1,3.4,16.27,9.05,0.35,-2.03
3,189.93733,19.90707,2614,350,54481,-23.37,45.132,0.001,42.55,0.02,15241.3,362.8,118.7,6.3,19.79,9.36,0.02,-2.33
4,189.99405,49.09448,1278,96,52735,-22.77,45.166,0.011,42.25,0.01,1385.6,48.6,52.5,2.9,15.96,7.29,0.03,-0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,231.08040,27.95578,1846,28,54173,-23.33,45.423,0.005,42.56,0.02,9345.2,348.3,60.6,3.4,24.76,9.08,0.03,-1.76
496,231.09229,26.70657,2160,444,53885,-22.72,44.989,0.005,42.19,0.04,4363.7,503.4,73.2,6.8,17.34,8.20,0.10,-1.31
497,231.11946,30.54376,1650,545,53174,-24.56,45.891,0.001,43.14,0.01,2572.5,106.9,76.1,1.9,40.15,8.19,0.04,-0.40
498,231.12225,35.25613,1354,466,52814,-22.60,45.143,0.005,41.80,0.05,1559.7,243.2,21.0,2.7,15.44,7.38,0.14,-0.34


Use astropy.cosmology for luminosity

## Bulge Luminosity of the Host Galaxy